In [1]:
instance_url = "https://lqrabuqwn0zjlu-1234.proxy.runpod.net/sdapi/v1/txt2img"

In [2]:
from random import choice as random_choice
from datetime import datetime
import requests
import base64
import re
import os
os.makedirs(f"_output", exist_ok=True)

In [3]:
def generate_savepath(payload, variant):
    def sanitize_filename(filename, replacement='_'):
        filename = re.sub(r'[<>:"/\\|?*]', replacement, filename) # Remove illegal characters
        filename = filename.strip(' .')                           # Remove leading/trailing whitespace and dots
        if filename.upper() in { "CON", "PRN", "AUX", "NUL", *(f"COM{i}" for i in range(1, 10)), *(f"LPT{i}" for i in range(1, 10)) }: filename = f'_{filename}' # Replace reserved names
        filename = filename[:250] # Optionally limit length (common safe limit is 255 characters)
        return filename
    prompt = payload["prompt"]
    model = payload["override_settings"]["sd_model_checkpoint"]
    curtime = datetime.now().strftime('%y%m%d%H%M%S')
    return f"""_output/{sanitize_filename(variant+"_"+model)}.jpg"""

payload = { "prompt": "amateur photo, a young man, toned body, (asian skin:1.5), topless, grey loose sweatpants, holding black phone, taking mirror selfie, plain dark wall, dark room, soft lighting, dark aesthetic, <lora:LORA_GHOST:0.75>, skeleton mask, call of duty, black hood", "negative_prompt": "", "styles": [], "seed": 1234567890, "subseed": -1, "subseed_strength": 0, "seed_resize_from_h": -1, "seed_resize_from_w": -1, "sampler_name": "DPM++ 2M SDE", "scheduler": "Karras", "batch_size": 1, "n_iter": 1, "steps": 20, "cfg_scale": 2.5, "distilled_cfg_scale": 3.5, "width": 896, "height": 1152, "restore_faces": False, "tiling": False, "do_not_save_samples": False, "do_not_save_grid": False, "denoising_strength": 0.7, "s_min_uncond": 0, "s_churn": 0, "s_tmax": 0, "s_tmin": 0, "s_noise": 1, "override_settings": {"sd_model_checkpoint": "LVSTIFY_V5.0", "CLIP_stop_at_last_layers": 1}, "override_settings_restore_afterwards": False, "disable_extra_networks": False, "comments": {}, "enable_hr": False, "hr_scale": 2, "hr_upscaler": "Latent", "hr_second_pass_steps": 0, "hr_resize_x": 0, "hr_resize_y": 0, "hr_additional_modules": ["Use same choices"], "hr_prompt": "", "hr_negative_prompt": "", "hr_cfg": 5, "hr_distilled_cfg": 3.5, "script_name": "", "script_args": [], "seed_enable_extras": True, "alwayson_scripts": {}, }
def imagen(payload, savepath, url):
    try:
        with requests.post(url=url, json=payload) as req:
            with open(savepath, "wb") as f:
                f.write(base64.b64decode(req.json()["images"][0]))
    except Exception as e:
        print(f"⚠️ Error > {e}")

In [4]:
prompt_variants = [
    "",
    "flat vector illustration",
    "flatline",
    "thick linework",
    "illustration",

    "flat color",
    "high contrast",
    "partially colored",
    "ligne claire",
    "limited palette",
    "no lineart",
    "lineart",
    "blending",
    "spot color",
    "color contrast",
    "outlines",
    "pastel colors",
    "pale color",
    "muted color",
    "color trace",
    "painterly",
    "colored lineart",
    "halftone",
    "screentones",
    "dithering",
    "analogous colors",
    "color coordination",
    "dispersion (optics)",
    "chromatic aberration",
    "film grain",
    "double exposure",
    "shade",
    "sidelighting",
    "backlighting",
    "underlighting",
    "silhouette",
    "vignetting",
    "watercolor effect",
    "inverted colors",

    "masterpiece",
    "best quality",
    "amazing quality",
]

base_prompt = "cowboy shot, solo, 1boy, dilf, orange hair, purple eyes, sad, crossed arms, looking at viewer, classroom, "

models = {
    "HSK_V2.2",
    "JNK_V4.0",
    "NVA_V8.0",
    # "PRF_V1.0",
    "WAI_V11.0",
    "WAI_V14.0",
}

for model in models:
    for variant in prompt_variants:
        payload["seed"] = 3367223820
        payload["sampler_name"] = "Euler a"
        payload["width"] = 832
        payload["height"] = 1216
        payload["override_settings"] = {"sd_model_checkpoint": model, "CLIP_stop_at_last_layers": 1}
        payload["prompt"] = base_prompt + variant
        payload["cfg_scale"] = 5.0
        payload["steps"] = 25
        imagen(payload, generate_savepath(payload, variant), instance_url)